<a href="https://colab.research.google.com/github/JimKing100/nfl-test/blob/master/Rookie_Kicker_Calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Imports
import pandas as pd

In [0]:
# Load the raw data
rookie_kicker_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-revised/rookies_non_kicker.csv')
kicking_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-revised/kicking.csv')
missed_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-revised/missedfg.csv')
conv_df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/nfl-test/master/data-revised/conv.csv')

In [3]:
# The dataframe of all 2019 non-rookie kickers and their rookie year
rookie_kicker_df.head()

,player,first,last,name,position1,start
0,AV-0400,Adam,Vinatieri,Adam Vinatieri,K,1996
1,MB-4600,Matt,Bryant,Matt Bryant,K,2002
2,RG-1500,Robbie,Gould,Robbie Gould,K,2005
3,MN-0800,Mike,Nugent,Mike Nugent,K,2005
4,SG-0800,Stephen,Gostkowski,Stephen Gostkowski,K,2006


In [4]:
# The kicking dataframe - raw kicking data for all 2019 kickers for seasons 2000-2019
kicking_df.head()

,uid,gid,player,del,first,last,name,position1,pat,fgs,fgm,fgl,fp,game,seas,year,team
0,15,8,AV-0400,KEEP,Adam,Vinatieri,Adam Vinatieri,K,1,1,0,0,4,1,5,2000,NE
1,59,30,AV-0400,KEEP,Adam,Vinatieri,Adam Vinatieri,K,1,4,0,0,13,2,5,2000,NE
2,76,38,AV-0400,KEEP,Adam,Vinatieri,Adam Vinatieri,K,1,0,0,0,1,3,5,2000,NE
3,100,51,AV-0400,KEEP,Adam,Vinatieri,Adam Vinatieri,K,0,0,1,0,4,4,5,2000,NE
4,122,63,AV-0400,KEEP,Adam,Vinatieri,Adam Vinatieri,K,4,0,0,0,4,5,5,2000,NE


In [5]:
# The missed field goal dataframe - raw missed field goal data for all 2019 kickers for seasons 2000-2019
missed_df.head()

,pid,gid,year,week,fgxp,fkicker,first,last,name,dist,good
0,517,4,2000,1,FG,JH-0500,John,Hall,John Hall,52,0
1,583,4,2000,1,FG,JH-0500,John,Hall,John Hall,48,0
2,877,6,2000,1,FG,KH-1500,Kris,Heppner,Kris Heppner,52,0
3,1203,8,2000,1,FG,AV-0400,Adam,Vinatieri,Adam Vinatieri,55,0
4,1435,9,2000,1,FG,DB-3900,Doug,Brien,Doug Brien,52,0


In [0]:
# Pull the stats to calculate the kicker points - the fp column is the points total for each game
def pull_stats(df, df1, df2, df3, col, p, y, g):
    missedfg_points = 0
    conversion_points = 0
    if (df[col].loc[(df['name']==p) & (df['year']==y) & (df['game']==g)].empty):
        start_year = df1['start'].loc[(df1['name']==p)].iloc[0]
        if y < start_year:
            points = float('NaN')
        else:
            points = 0
    else:
        conversion_points1 = df3['conv'].loc[(df3['name_bc']==p) & (df3['year']==y) & (df3['week']==g)].count()
        conversion_points2 = df3['conv'].loc[(df3['name_psr']==p) & (df3['year']==y) & (df3['week']==g)].count()
        conversion_points3 = df3['conv'].loc[(df3['name_trg']==p) & (df3['year']==y) & (df3['week']==g)].count()

        conversion_points = conversion_points1 + conversion_points2 + conversion_points3
        missedfg_points = df2['good'].loc[(df2['name']==p) & (df2['year']==y) & (df2['week']==g)].count()
        points = df[col].loc[(df['name']==p) & (df['year']==y) & (df['game']==g)].iloc[0]

    return points - missedfg_points + conversion_points


In [0]:
# Add a row to the final_df dataframe
# Each row represents the actual kicking points for each game for each kicker
def add_row(df, p, f, l, n, pos, value_list):

    df = df.append({'player': p,
                    'first': f,
                    'last': l,
                    'name': n,
                    'position1': pos,
                    'game1': value_list[0], 
                    'game2': value_list[1],
                    'game3': value_list[2], 
                    'game4': value_list[3], 
                    'game5': value_list[4], 
                    'game6': value_list[5], 
                    'game7': value_list[6], 
                    'game8': value_list[7], 
                    'game9': value_list[8],
                    'game10': value_list[9],
                    'game11': value_list[10],
                    'game12': value_list[11],
                    'game13': value_list[12],
                    'game14': value_list[13],
                    'game15': value_list[14],
                    'game16': value_list[15]
                    }, ignore_index=True)
    
    return df

In [0]:
# The main code for iterating through the kicker list, calculating the points and adding the rows
# to the final_df dataframe.
column_names = ['player',
                'first',
                'last',
                'name',
                'position1', 
                '2000-game1',
                '2000-game2',
                '2000-game3', 
                '2000-game4', 
                '2000-game5', 
                '2000-game6', 
                '2000-game7', 
                '2000-game8', 
                '2000-game9',
                '2000-game10',
                '2000-game11',
                '2000-game12',
                '2000-game13',
                '2000-game14',
                '2000-game15',
                '2000-game16'
                ]

kickers_df = player_df.loc[player_df['position1'] == 'K']
player_list = kickers_df['name'].tolist()

final_df = pd.DataFrame(columns = column_names)

for player in player_list:

    first = player_df['first'].loc[(player_df['name']==player)].iloc[0]
    last = player_df['last'].loc[(player_df['name']==player)].iloc[0]
    name = player_df['name'].loc[(player_df['name']==player)].iloc[0]
    position1 = player_df['position1'].loc[(player_df['name']==player)].iloc[0]

    player_scores = []

    for year in range(2000,2020):
        for game in range(1,17):
            score = pull_stats(kicking_df, player_df, missed_df, conv_df, 'fp', player, year, game)

            player_scores.append(score)

    final_df = add_row(final_df, player, first, last, name, position1, player_scores)